In [2]:
# Load environment variables and set up auto-reload
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload

In [3]:
import sys
sys.path.append('../')

In [4]:
from src.utils import show_prompt
from src.prompt import forwarder_agent_tasks
show_prompt(forwarder_agent_tasks, "forwarder_agent_tasks")

c:\Users\kamalmo\Desktop\AI\InboundLogisticSystem\MultiAgentInboundLogistic\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


╭───────────────────────────────────────────── forwarder_agent_tasks ─────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [1]:
%%writefile ../src/ibl_data_source.py

"""
   This data source represent the IBL Schema
"""
import json

def ibl_data_source(path: str , agent: str) -> dict[str : str]:
    try:
        with open(path, "r", encoding="utf-8") as config:
            config_file = json.load(config).get(agent, [])
    except FileNotFoundError:
        print("Error: logistics_schema.json not found. Please create it.")
        exit()
    return config_file

Overwriting ../src/ibl_data_source.py


In [ ]:
%%writefile ../src/forwarder_schema.py

"""State Definitions and Pydantic Schemas for Forwarder Agent.

This defines the state objects and structured schemas used for
the Forwarder Agent scoping workflow, including Forwarder state management and output schemas.
"""

import operator
from datetime import date
from typing_extensions import Optional, Annotated, List, Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph import MessagesState
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field , create_model
from src.supervisor_schema import AgentState
from src.ibl_data_source import ibl_data_source

# Load logitics fields dynamiclly
logistics_fields = ibl_data_source("../ibl_schema.json","logistics_agent")

# Dynamically create Pydantic model for shipment fields
DynamicShipmentFields = create_model(
    "DynamicShipmentFields",
    **{
        field_item["field"]: (
            Optional[field_item["dataType"]],  # default type; could later map dataType
            Field(None, description = field_item.get("description", ""))
        )
        for field_item in logistics_fields
    }
)

# ===== STRUCTURED OUTPUT SCHEMAS =====

class ForwarderSchema(BaseModel):
    """Schema for Logisticis Agent."""
    missing_mandatory_fields: List[str] = Field(
        description = "Fields required by the schema that are missing from the provided data"
    )
    missing_optional_fields: List[str] = Field(
        description = "Optional fields that are missing from the provided data"
    )
    ask_for_optional_fields: bool = Field(
        description = "Specifies whether the user should be prompted for optional fields",
        default = True
    )
    needs_user_confirmation: bool = Field(
        description = "Specifies whether user confirmation is required for the current record",
        default = True
    )

# ===== STATE DEFINITIONS =====

class ForwarderState(AgentState):
    """ State for the Logistics Agent """
    agent_response: Optional[LogisticsSchema] = None